In [5]:
import albumentations as A
import cv2
import torch



In [7]:
# 1. Define the augmentation pipeline

transform = A.Compose([
    A.Resize(width=224, height=224, always_apply=True),
    A.Normalize(max_pixel_value=255.0, always_apply=True),
])

# 2. Load an image using OpenCV (Note: Albumentations expects RGB)

IMAGE_FILE='../../03-datasets/03-flickr-8k/data/Images/667626_18933d713e.jpg'

image = cv2.imread(IMAGE_FILE)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 3. Apply the transformations
# The result is a dictionary; we grab the 'image' key
transformed = transform(image=image)
transformed_image = transformed["image"]

# 4. convert numpy array to pytorch tensor
transformed_tensor = torch.from_numpy(transformed_image)

# 5. Prepare the tensor for ResNet50: expected shape [1, 3, 224, 224] and dtype float, normalized to [0,1]
# Permute dimensions from [H, W, C] to [C, H, W]
resnet_input = transformed_tensor.permute(2, 0, 1).unsqueeze(0).float()
resnet_input.shape



/var/folders/s7/4mzvpqd570j7jzsl_gsb_rnc0000gn/T/ipykernel_86536/2779179732.py:4: UserWarning: Argument(s) 'always_apply' are not valid for transform Resize
  A.Resize(width=224, height=224, always_apply=True),
/var/folders/s7/4mzvpqd570j7jzsl_gsb_rnc0000gn/T/ipykernel_86536/2779179732.py:5: UserWarning: Argument(s) 'always_apply' are not valid for transform Normalize
  A.Normalize(max_pixel_value=255.0, always_apply=True),


torch.Size([1, 3, 224, 224])